In [20]:
import cvxpy as cp
import numpy as np
from basis_generator import generate_basis, sel_seq

In [16]:
dim = 2
num_obs = 5
len_seq = 2
out_max = 1
basis_size= 242
seq_method="sel_sequences"

KCBS_events = [((0, 1), (0, 1)),
            ((0, 1), (1, 2)),
            ((0, 1), (2, 3)),
            ((0, 1), (3, 4)),
            ((0, 1), (4, 0)),
            ((1, 0), (0, 1)),
            ((1, 0), (1, 2)),
            ((1, 0), (2, 3)),
            ((1, 0), (3, 4)),
            ((1, 0), (4, 0)),
                ]

In [17]:
X_basis, rank = generate_basis(dim = dim,
                num_obs = num_obs,
                len_seq = len_seq,
                out_max = out_max,
                batch_size = basis_size,
                seq_method= seq_method,
                sel_sequences = [len_seq],
                remove_last_out = False)

In [22]:
alpha = cp.Variable((len(X_basis), 1))
X = cp.Variable(X_basis[0].shape, symmetric=True)

constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
constraints += [
    cp.trace(X) == num_obs**len_seq
]

In [23]:
sequences = sel_seq(num_obs = num_obs,
                len_seq = len_seq,
                out_max = out_max,
                remove_last_out = False)

C = np.zeros(X_basis[0].shape)
for i, seq_row in enumerate(sequences):
    if seq_row in KCBS_events:
        C[i,i] = 1
        
prob = cp.Problem(cp.Maximize(cp.trace(C@X)),
                  constraints)

In [24]:
prob.solve(solver=cp.MOSEK, verbose=False)

print(prob.status)
print("The optimal value is", prob.value)

optimal
The optimal value is 4.522542433953802
